# How Manualy Hyper Parameter Tuning work

In [132]:
from sklearn.datasets import load_iris
iris = load_iris()
dir(iris)

['DESCR',
 'data',
 'data_module',
 'feature_names',
 'filename',
 'frame',
 'target',
 'target_names']

In [133]:
import pandas as pd
df = pd.DataFrame(iris.data, columns=iris.feature_names)
df["flower"] = iris.target
df["flower"] = df["flower"].apply(lambda x : iris.target_names[x])
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),flower
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [134]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.2)

In [135]:
from sklearn.svm import SVC
svc = SVC()

svc.fit(X_train, y_train)
svc.score(X_test, y_test)

0.9333333333333333

With train_test_split() our score may change every time. Thats why we use cross_val_score()

In [136]:
from sklearn.model_selection import cross_val_score
from sklearn import svm 

In [137]:
cross_val_score(svm.SVC(kernel="linear", C=10, gamma="auto"), iris.data, iris.target, cv=5)

array([1.        , 1.        , 0.9       , 0.96666667, 1.        ])

In [138]:
cross_val_score(svm.SVC(kernel="rbf", C=20, gamma="auto"), iris.data, iris.target, cv=5)

array([0.96666667, 1.        , 0.9       , 0.96666667, 1.        ])

In [139]:
cross_val_score(svm.SVC(kernel="rbf", C=30, gamma="auto"), iris.data, iris.target, cv=5)

array([0.96666667, 1.        , 0.9       , 0.93333333, 1.        ])

# Hyper Parameter Tuning using for loop

In [140]:
import numpy as np

kernels = ["rbf", "linear"]
c = [10, 20, 30]
scores = {}

for k in kernels:
    for C in c:
        score = cross_val_score(svm.SVC(gamma="auto", kernel=k, C=C), iris.data, iris.target, cv=5)
        scores[f"SVC(kernel={k}, C={C})"] = np.average(score)

scores

{'SVC(kernel=rbf, C=10)': np.float64(0.9800000000000001),
 'SVC(kernel=rbf, C=20)': np.float64(0.9666666666666668),
 'SVC(kernel=rbf, C=30)': np.float64(0.96),
 'SVC(kernel=linear, C=10)': np.float64(0.9733333333333334),
 'SVC(kernel=linear, C=20)': np.float64(0.9666666666666666),
 'SVC(kernel=linear, C=30)': np.float64(0.96)}

# GridSearchCV() can simplify those all manual tasks

In [141]:
from sklearn.model_selection import GridSearchCV

clf = GridSearchCV(svm.SVC(gamma="auto"),{
    "C" : [10,20,30],
    "kernel" : ["linear", "rbf"]
}, cv=5, return_train_score=False)

In [142]:
clf.fit(iris.data, iris.target)
clf.cv_results_

{'mean_fit_time': array([0.00120006, 0.00082583, 0.0008604 , 0.0007977 , 0.00084386,
        0.00074291]),
 'std_fit_time': array([1.64394795e-04, 5.39305566e-05, 1.27548583e-04, 4.12615434e-05,
        1.19500720e-04, 1.34810897e-05]),
 'mean_score_time': array([0.00077457, 0.00054884, 0.0005506 , 0.00051398, 0.00054293,
        0.00047708]),
 'std_score_time': array([1.55882319e-04, 3.76749639e-05, 8.16025125e-05, 1.91338300e-05,
        8.33605460e-05, 6.30255354e-06]),
 'param_C': masked_array(data=[10, 10, 20, 20, 30, 30],
              mask=[False, False, False, False, False, False],
        fill_value=999999),
 'param_kernel': masked_array(data=['linear', 'rbf', 'linear', 'rbf', 'linear', 'rbf'],
              mask=[False, False, False, False, False, False],
        fill_value=np.str_('?'),
             dtype=object),
 'params': [{'C': 10, 'kernel': 'linear'},
  {'C': 10, 'kernel': 'rbf'},
  {'C': 20, 'kernel': 'linear'},
  {'C': 20, 'kernel': 'rbf'},
  {'C': 30, 'kernel': 'line

In [143]:
import pandas as pd

df = pd.DataFrame(clf.cv_results_)
df.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.001200,0.000164,0.000775,0.000156,10,linear,"{'C': 10, 'kernel': 'linear'}",1.000000,1.0,0.900000,0.966667,1.0,0.973333,0.038873,2
1,0.000826,0.000054,0.000549,0.000038,10,rbf,"{'C': 10, 'kernel': 'rbf'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
2,0.000860,0.000128,0.000551,0.000082,20,linear,"{'C': 20, 'kernel': 'linear'}",1.000000,1.0,0.900000,0.933333,1.0,0.966667,0.042164,4
3,0.000798,0.000041,0.000514,0.000019,20,rbf,"{'C': 20, 'kernel': 'rbf'}",0.966667,1.0,0.900000,0.966667,1.0,0.966667,0.036515,3
4,0.000844,0.000120,0.000543,0.000083,30,linear,"{'C': 30, 'kernel': 'linear'}",1.000000,1.0,0.900000,0.900000,1.0,0.960000,0.048990,5


In [144]:
df = df[["params", "mean_test_score", "rank_test_score"]]
df.head()

,params,mean_test_score,rank_test_score
0,"{'C': 10, 'kernel': 'linear'}",0.973333,2
1,"{'C': 10, 'kernel': 'rbf'}",0.980000,1
2,"{'C': 20, 'kernel': 'linear'}",0.966667,4
3,"{'C': 20, 'kernel': 'rbf'}",0.966667,3
4,"{'C': 30, 'kernel': 'linear'}",0.960000,5


In [145]:
dir(clf)

['__abstractmethods__',
 '__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__sklearn_clone__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_build_request_for_signature',
 '_check_feature_names',
 '_check_n_features',
 '_check_refit_for_multimetric',
 '_doc_link_module',
 '_doc_link_template',
 '_doc_link_url_param_generator',
 '_estimator_type',
 '_format_results',
 '_get_default_requests',
 '_get_doc_link',
 '_get_metadata_request',
 '_get_param_names',
 '_get_routed_params_for_fit',
 '_get_scorers',
 '_get_tags',
 '_more_tags',
 '_parameter_constraints',
 '_repr_html_',
 '_repr_html_inner',
 '_repr_mimebundle_',
 '_required_parameters',
 '_run

In [146]:
clf.best_score_

np.float64(0.9800000000000001)

In [147]:
clf.best_params_

{'C': 10, 'kernel': 'rbf'}

# GridSearchCV() is best to handel all manual tasks. But still we have computational limitation and to make our Hyper Parameter Tuning efficient for computation we have RandomizedSearchCV()

In [148]:
from sklearn.model_selection import RandomizedSearchCV

clf = RandomizedSearchCV(svm.SVC(gamma="auto"), {
    "C" : [10, 20, 30],
    "kernel" : ["linear", "rbf"]
}, cv=5, return_train_score=False, n_iter=2)

In [149]:
clf.fit(iris.data, iris.target)

RandomizedSearchCV(cv=5, estimator=SVC(gamma='auto'), n_iter=2,
                   param_distributions={'C': [10, 20, 30],
                                        'kernel': ['linear', 'rbf']})

In [150]:
df = pd.DataFrame(clf.cv_results_)
df.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_kernel,param_C,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.001257,0.000118,0.000924,0.000149,rbf,10,"{'kernel': 'rbf', 'C': 10}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
1,0.001067,0.000127,0.000699,0.000076,linear,10,"{'kernel': 'linear', 'C': 10}",1.000000,1.0,0.900000,0.966667,1.0,0.973333,0.038873,2


In [151]:
df = df[["param_kernel", "param_C", "mean_test_score", "rank_test_score"]]
df.head()

,param_kernel,param_C,mean_test_score,rank_test_score
0,rbf,10,0.980000,1
1,linear,10,0.973333,2
